In [90]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

# the following line is not required if BatchFlow is installed as a python package.
sys.path.append('../../../batchflow')
from batchflow import B, V, F, R, P, C, Dataset, Batch, action, inbatch_parallel, ImagesBatch
from batchflow.models.torch import TorchModel
from batchflow.models.metrics import ClassificationMetrics
from batchflow.opensets import MNIST
from batchflow import best_practice

In [91]:
BATCH_SIZE = 64
mnist = MNIST()

Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-labels-idx1-ubyte.gz
DownloadingExtracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-images-idx3-ubyte.gz
 http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-images-idx3-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-labels-idx1-ubyte.gz


In [92]:
class RegressionModel(TorchModel):

    @classmethod
    def default_config(cls):
        config = TorchModel.default_config()
        config['head'].update(dict(layout='f'))
        return config

In [93]:
model_config = {
    'inputs': dict(images={'shape': (28, 28, 1)},
                   labels={'classes': 10}),
    'loss': 'ce',
    'optimizer': 'Adam',
    'initial_block/inputs': 'images',
    'head/units': 10,
    'output': dict(predicted=['proba', 'labels'])
}

In [94]:
data_dict = {
    'images': B('images'),
    'labels': B('labels')
}

In [96]:
train_pipeline = (mnist.train.p
                .init_variable('predictions')
                .init_variable('loss_history', init_on_each_run=list)
                .init_model('dynamic',
                            RegressionModel,
                            'logistic',
                            config=model_config)
                .train_model('logistic',
                             fetches='predictions',
                             feed_dict=data_dict,
                             save_to=V('predictions'))
                .gather_metrics(ClassificationMetrics, targets=B('labels'), predictions=V('predictions'),
                    fmt='logits', axis=-1, save_to=V('metrics'), mode='a'))

In [ ]:
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=False, bar=True, prefetch=1)










  0%|          | 0/938 [00:00<?, ?it/s]  File "../../../batchflow/batchflow/pipeline.py", line 1018, in _run_batches_from_queue
    batch = future.result()
  File "/anaconda3/lib/python3.6/concurrent/futures/_base.py", line 432, in result
    return self.__get_result()
  File "/anaconda3/lib/python3.6/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/anaconda3/lib/python3.6/concurrent/futures/thread.py", line 56, in run
    result = self.fn(*self.args, **self.kwargs)
  File "../../../batchflow/batchflow/pipeline.py", line 609, in execute_for
    batch_res = self._exec_all_actions(batch)
  File "../../../batchflow/batchflow/pipeline.py", line 573, in _exec_all_actions
    action_fn(batch, _action)
  File "../../../batchflow/batchflow/pipeline.py", line 887, in _exec_train_model
    output = model.train(*args, **kwargs)


Exception in a thread: train() got an unexpected keyword argument 'feed_dict'
